# Loop 22 Analysis: Remaining Approaches

## Current State
- **Best LB**: 0.0913 (exp_012 - MLP[32,16] + LightGBM ensemble)
- **Best CV**: 0.008785 (exp_011)
- **Target**: 0.0333
- **Gap**: 2.74x to target

## Recent Experiments (Both Negative)
1. **Attention Model (exp_021)**: CV 0.023357 - 159% WORSE than baseline
2. **Fragprints (exp_020)**: CV 0.009749 - 8.28% WORSE than baseline

## Remaining Unexplored Approaches
1. **ACS PCA features** (5 features) - NOT YET TRIED
2. **Per-target models** - Separate models for Product 2, Product 3, SM
3. **Stacking** - Meta-learner on out-of-fold predictions
4. **LightGBM hyperparameter tuning**
5. **Polynomial features** - Interaction terms

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load data
DATA_PATH = '/home/data'

# Load all feature files
spange_df = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
drfp_df = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
acs_pca_df = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)
fragprints_df = pd.read_csv(f'{DATA_PATH}/fragprints_lookup.csv', index_col=0)

print('Feature dimensions:')
print(f'  Spange: {spange_df.shape}')
print(f'  DRFP: {drfp_df.shape}')
print(f'  ACS PCA: {acs_pca_df.shape}')
print(f'  Fragprints: {fragprints_df.shape}')

In [ ]:
# Analyze ACS PCA features
print('\n=== ACS PCA DESCRIPTORS ===')
print(acs_pca_df.head(10))
print(f'\nSolvents in ACS PCA: {len(acs_pca_df)}')
print(f'Solvents in Spange: {len(spange_df)}')
print(f'Solvents in DRFP: {len(drfp_df)}')

# Check overlap
acs_solvents = set(acs_pca_df.index)
spange_solvents = set(spange_df.index)
drfp_solvents = set(drfp_df.index)

print(f'\nOverlap ACS PCA & Spange: {len(acs_solvents & spange_solvents)}')
print(f'Overlap ACS PCA & DRFP: {len(acs_solvents & drfp_solvents)}')
print(f'Overlap all three: {len(acs_solvents & spange_solvents & drfp_solvents)}')

In [ ]:
# Check which solvents are in single solvent data
single_df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
full_df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')

single_solvents = set(single_df['SOLVENT NAME'].unique())
full_solvents_a = set(full_df['SOLVENT A NAME'].unique())
full_solvents_b = set(full_df['SOLVENT B NAME'].unique())
all_data_solvents = single_solvents | full_solvents_a | full_solvents_b

print(f'\nSolvents in single data: {len(single_solvents)}')
print(f'Solvents in full data (A): {len(full_solvents_a)}')
print(f'Solvents in full data (B): {len(full_solvents_b)}')
print(f'All unique solvents in data: {len(all_data_solvents)}')

# Check if ACS PCA covers all data solvents
missing_in_acs = all_data_solvents - acs_solvents
print(f'\nSolvents missing in ACS PCA: {missing_in_acs}')
if missing_in_acs:
    print('WARNING: ACS PCA does not cover all solvents!')

In [ ]:
# Analyze correlation between ACS PCA and Spange
common_solvents = list(acs_solvents & spange_solvents)
print(f'\n=== CORRELATION ANALYSIS ===')
print(f'Common solvents: {len(common_solvents)}')

# Get aligned features
acs_aligned = acs_pca_df.loc[common_solvents]
spange_aligned = spange_df.loc[common_solvents]

# Compute correlation matrix between ACS PCA and Spange
from scipy.stats import pearsonr

print('\nCorrelation between ACS PCA and Spange features:')
for acs_col in acs_pca_df.columns:
    max_corr = 0
    max_spange_col = None
    for spange_col in spange_df.columns:
        corr, _ = pearsonr(acs_aligned[acs_col], spange_aligned[spange_col])
        if abs(corr) > abs(max_corr):
            max_corr = corr
            max_spange_col = spange_col
    print(f'  {acs_col}: max corr {max_corr:.3f} with {max_spange_col}')

In [ ]:
# Analyze per-target performance from exp_012
print('\n=== PER-TARGET ANALYSIS ===')
print('Looking at exp_012 results to understand per-target performance...')

# Load single solvent data to analyze target distributions
Y = single_df[['Product 2', 'Product 3', 'SM']]
print(f'\nTarget statistics (single solvent):')
print(Y.describe())

# Check correlation between targets
print(f'\nTarget correlations:')
print(Y.corr())

In [ ]:
# Analyze CV-LB relationship
print('\n=== CV-LB RELATIONSHIP ===')
submissions = [
    ('exp_000', 0.0111, 0.0982),
    ('exp_001', 0.0123, 0.1065),
    ('exp_003', 0.0105, 0.0972),
    ('exp_005', 0.0104, 0.0969),
    ('exp_006', 0.0097, 0.0946),
    ('exp_007', 0.0093, 0.0932),
    ('exp_009', 0.0092, 0.0936),
    ('exp_012', 0.0090, 0.0913),
]

cv_scores = [s[1] for s in submissions]
lb_scores = [s[2] for s in submissions]

# Linear fit
from scipy.stats import linregress
slope, intercept, r_value, p_value, std_err = linregress(cv_scores, lb_scores)
print(f'Linear fit: LB = {slope:.2f} * CV + {intercept:.4f}')
print(f'R² = {r_value**2:.4f}')
print(f'\nTo reach target LB 0.0333:')
required_cv = (0.0333 - intercept) / slope
print(f'  Required CV: {required_cv:.6f}')
if required_cv < 0:
    print('  WARNING: Target requires negative CV - mathematically impossible with current approach!')

In [ ]:
# Summary of remaining approaches
print('\n=== REMAINING APPROACHES TO TRY ===')
print()
print('1. ACS PCA FEATURES (5 features)')
print('   - Status: NOT YET TRIED')
print('   - Covers 24 solvents (same as DRFP)')
print('   - Low correlation with Spange - provides NEW information')
print('   - Quick experiment: add to existing feature set')
print()
print('2. PER-TARGET MODELS')
print('   - Status: NOT YET TRIED')
print('   - Targets have different distributions (SM is inverse of products)')
print('   - Competition rules explicitly allow different hyperparams per target')
print('   - Could train separate MLP+LGBM for each of Product 2, Product 3, SM')
print()
print('3. STACKING')
print('   - Status: NOT YET TRIED')
print('   - Current: fixed weights (0.6 MLP, 0.4 LGBM)')
print('   - Could train meta-learner on out-of-fold predictions')
print()
print('4. LIGHTGBM HYPERPARAMETER TUNING')
print('   - Status: NOT YET TRIED')
print('   - Current params may not be optimal')
print('   - Quick grid search could help')
print()
print('5. POLYNOMIAL FEATURES')
print('   - Status: NOT YET TRIED')
print('   - Interaction terms between Spange descriptors')
print('   - Could capture non-linear relationships')